In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped_processed/"+'melody.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries_processed/"+'melody.json'


In [2]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [3]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [4]:
from mongeau_sankoff import similarity, Note
import miditoolkit as mtk
# queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)

In [25]:
#obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_melody.mid")
obj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_clipped_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_melody.mid")


In [26]:
notes = obj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t= ans
        


In [27]:
rep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        rep.append(Note("C",int((n.start-prev_end)/t),rest=True))
    rep.append(Note(midiEt_to_note[n.pitch%12 + 12],int((n.end-n.start)/t),rest=False))
    prev_end = n.end

In [32]:
qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1_query_melody.mid")
#qobj = mtk.MidiFile(r"C:\Users\Kshitij Alwadhi\Documents\GitHub\Clarinet\Data\Melody\2018_queries_processed\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1_query_melody.mid")


In [33]:
notes = qobj.instruments[0].notes
end_time = notes[-1].end
tempo = 120
t = end_time/(4*tempo)

ans = 99999
prev_end = 0
for n in notes:
    if n.start > prev_end:
        dur = n.start - prev_end
        ans = min(ans, dur)
    ans = min(ans, n.end - n.start)
    prev_end = n.end
t = ans


In [34]:
qrep = []
prev_end = 0
for n in notes:
    if n.start > prev_end:
        qrep.append(Note("C", int((n.start-prev_end)/t), rest=True))
    qrep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False))
    prev_end = n.end


In [31]:
similarity(qrep, rep)

40it [00:00, 169.49it/s]


-26.855666666666664

In [35]:
from tqdm import tqdm
import os
import music21
fnames = os.listdir("C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/")


def getKey(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.tonic.name)
def getMode(midi_filename):
    score = music21.converter.parse(midi_filename)
    key = score.analyze('key')
    return(key.mode)


In [36]:
def compute_similarities(qrep):
    fname_to_similarity = {}
    for fname in fnames:
        sim = 0
        obj = mtk.MidiFile(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        notes = obj.instruments[0].notes 
        ans = 99999
        prev_end = 0
        
        for n in notes:
            if n.start > prev_end:
                dur = n.start - prev_end
                ans = min(ans, dur)
            ans = min(ans, n.end - n.start)
            prev_end = n.end
        t = ans
        mode = getMode(r"C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Melody/2018_clipped_processed/" + fname)
        
        rep = []
        prev_end = 0
        for n in notes:
            if n.start > prev_end:
                rep.append(Note("C", int((n.start-prev_end)/t), rest=True,scale=mode))
            rep.append(Note(midiEt_to_note[n.pitch % 12 + 12], int((n.end-n.start)/t), rest=False,scale=mode))
            prev_end = n.end
        sim = similarity(qrep, rep)
        fname_to_similarity[fname] = sim
    return fname_to_similarity


In [37]:
fname_to_similarity = compute_similarities(qrep)

61it [00:04, 12.66it/s]
13it [00:01, 12.21it/s]
7it [00:00, 10.67it/s]
34it [00:02, 12.62it/s]
51it [00:03, 15.14it/s]
26it [00:02, 10.24it/s]
54it [00:05, 10.47it/s]
67it [00:05, 12.02it/s]
72it [00:06, 11.91it/s]
28it [00:02, 10.66it/s]
103it [00:08, 12.25it/s]
24it [00:02, 10.04it/s]
77it [00:07,  9.82it/s]
44it [00:03, 11.55it/s]
37it [00:03, 12.00it/s]
110it [00:08, 12.28it/s]
















15it [00:02,  7.42it/s]




119it [00:30, 13.34it/s]




































































































































































































































































































































































































































































































































































In [16]:
dict(sorted(fname_to_similarity.items(), key=lambda item: item[1], reverse=True))

{'MIDI-Unprocessed_Schubert10-12_MID--AUDIO_17_R2_2018_wav_melody.mid': 0.5650000000000002,
 'MIDI-Unprocessed_Schubert7-9_MID--AUDIO_15_R2_2018_wav_melody.mid': 0.5070000000000001,
 'MIDI-Unprocessed_Schubert10-12_MID--AUDIO_18_R2_2018_wav_melody.mid': 0.3910000000000001,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--3_melody.mid': 0.246,
 'MIDI-Unprocessed_Schubert4-6_MID--AUDIO_08_R2_2018_wav_melody.mid': 0.21699999999999997,
 'MIDI-Unprocessed_Recital5-7_MID--AUDIO_05_R1_2018_wav--2_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital9-11_MID--AUDIO_09_R1_2018_wav--4_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital9-11_MID--AUDIO_09_R1_2018_wav--6_melody.mid': 0.18799999999999983,
 'MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--1_melody.mid': 0.15899999999999992,
 'MIDI-Unprocessed_Recital1-3_MID--AUDIO_02_R1_2018_wav--1_melody.mid': 0.1299999999999999,
 'MIDI-Unprocessed_Recital4_MID--AUDIO_04_R1_2018_wav--4_melody.mid': 0.10099999999999